ogólnie to nie działa

# Initialization & params

In [194]:
import pandas as pd
import numpy as np

from datetime import datetime as dt
from functools import partial, wraps

import plasticc.xgb_train as xgb
import plasticc.lgbm_train as lgbm
from plasticc.featurize import process_meta
from plasticc.featurize import featurize
from plasticc.lgbm_train import lgbm_modeling_cross_validation
from plasticc.xgb_train import xgb_modeling_cross_validation
from plasticc.final import featurize_test, predict_test

In [195]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


UsageError: Missing module name.


In [204]:
%reload_ext plasticc.lgbm_train

In [9]:
fcp = {
    'flux': {
        'longest_strike_above_mean': None,
        'longest_strike_below_mean': None,
        'mean_change': None,
        'mean_abs_change': None,
        'length': None,
    },

    'flux_by_flux_ratio_sq': {
        'longest_strike_above_mean': None,
        'longest_strike_below_mean': None,       
    },

    'flux_passband': {
        'fft_coefficient': [
                {'coeff': 0, 'attr': 'abs'}, 
                {'coeff': 1, 'attr': 'abs'}
            ],
        'kurtosis' : None, 
        'skewness' : None,
    },

    'mjd': {
        'maximum': None, 
        'minimum': None,
        'mean_change': None,
        'mean_abs_change': None,
    },
}

In [10]:
aggs = {
    'flux': ['min', 'max', 'mean', 'median', 'std', 'skew'],
    'flux_err': ['min', 'max', 'mean', 'median', 'std', 'skew'],
    'detected': ['mean'],
    'flux_ratio_sq':['sum', 'skew'],
    'flux_by_flux_ratio_sq':['sum','skew'],
}

In [11]:
# calculate test size
!wc -l ../data/raw/test_set.csv

453653105 ../data/raw/test_set.csv


In [12]:
chunk_size_one_100 = 453653105 // 100 + 1
#chunk_size_one_100 = 1000001 // 100 + 1
chunk_size_one_100

4536532

### Getting features from test set

In [13]:
already_featurized = True
if not already_featurized:
    featurize_test(
        featurize_configs={'aggs': aggs, 'fcp': fcp}, 
        n_jobs=4,
        meta_path='../data/raw/test_set_metadata.csv',
        test_path='../data/raw/test_set.csv',
       output_path='feat_test.csv',
        id_colname='object_id',
        chunks=5000000,
    )

# Training data preparing

In [154]:
%%time
meta_train = process_meta('../data/raw/training_set_metadata.csv')
train = pd.read_csv('../data/raw/training_set.csv')
print(train.shape)
print(meta_train.shape)

(1421705, 6)
(7848, 15)
CPU times: user 892 ms, sys: 880 ms, total: 1.77 s
Wall time: 1.77 s


In [238]:
%%time
X = featurize(train, meta_train, aggs, fcp, n_jobs=4)
X_backup = X.copy()
Xcols = X.columns
print(X.shape)

Feature Extraction: 100%|██████████| 20/20 [00:01<00:00, 13.21it/s]


(7848, 70)
CPU times: user 26.2 s, sys: 8.44 s, total: 34.7 s
Wall time: 42.1 s


In [389]:
X = X_backup.copy()

if 'target' in X:
    y = X['target']
    del X['target']
else:
    print("What the duck")
    3//0
if 'object_id' in X:
    del X['object_id']
else:
    print("What the heck")
    3//0
print(X.shape)

Xcols = X.columns

X = X.fillna(0)
X = StandardScaler().fit_transform(X)
X = pd.DataFrame(data=X, columns=Xcols)

(7848, 68)


# Autoencoder

In [351]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler, StandardScaler

import torch
from torch import nn
from torch.autograd import Variable

### Preparing featurized test set to training

In [ ]:
Xinp = pd.read_csv('feat_test.csv').drop('object_id', axis=1)
Xinp = np.nan_to_num(Xinp.values)
Xinp = StandardScaler().fit_transform(Xinp)
print(Xinp.shape)

### Splitting test set to batches

In [390]:
batch_size = 5

#batch_num = Xinp.shape[0]//batch_size
#Xs = [Xinp[i*batch_size : (i+1)*batch_size] for i in range(0, batch_num)]
batch_num = X.shape[0]//batch_size
Xs = [X[i*batch_size : (i+1)*batch_size] for i in range(0, batch_num)]

assert batch_num == len(Xs)
assert all([s.shape[0] == batch_size for s in Xs])

### Autoencoder definition

In [354]:
learning_rate = 0.001
num_epochs = 10

In [377]:
class autoencoder(nn.Module):
    def __init__(self, input_size, latent_size):
        super(autoencoder, self).__init__()

        self.train_mode = True
        
        if input_size <= latent_size + 2:
            raise Exception("input size is not enough bigger than latent size")
        
        self.input_size = input_size
        self.latent_size = latent_size
        self.mid_size = (input_size + latent_size) // 2 

        self.encoder = nn.Sequential(
            nn.Linear(input_size, self.mid_size),
            nn.ReLU(True),
            nn.Linear(self.mid_size, latent_size)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_size, self.mid_size),
            nn.ReLU(True),
            nn.Linear(self.mid_size, input_size),
        )

    def forward(self, x):
        x = self.encoder(x)
        if self.train_mode:
            x = self.decoder(x)
        return x
    
    def encode_mode(self, b=False):
        self.train_mode=b

In [391]:
#model_savior = 2//0  # To not remove it accidentally

model = autoencoder(input_size=Xs[0].shape[1],
                    latent_size=30
                   ).cuda(2)
criterion = nn.MSELoss()
optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate,
                             weight_decay=1e-5)

### Training autoencoder and setting it to encoding mode

In [392]:
#model_savior = 2//0  # To not remove it accidentally

model.train()
for epoch in range(num_epochs):
    for X_ in Xs:
        X_ = Variable(torch.tensor(X_.values.astype(np.float32))).cuda(2)
        # ===================forward=====================
        output = model(X_)
        loss = criterion(output, X_)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    if epoch % (num_epochs//10) == 0 or epoch == num_epochs - 1:
        print('epoch [{}/{}], loss:{:.4f}'
              .format(epoch+1, num_epochs, loss.data.item()))
model.eval()
model.encode_mode()

epoch [1/10], loss:0.8257
epoch [2/10], loss:0.8202
epoch [3/10], loss:0.8154
epoch [4/10], loss:0.8111
epoch [5/10], loss:0.8071
epoch [6/10], loss:0.8034
epoch [7/10], loss:0.7999
epoch [8/10], loss:0.7965
epoch [9/10], loss:0.7932
epoch [10/10], loss:0.7901


# Training data processing

In [393]:
def runModel(X):
    X = model(X)
    X = pd.DataFrame(data=X.detach().cpu().numpy(), 
                 columns=['aut' + str(i) for i in range(0, X.shape[1])])
    return X

In [394]:
Xv = Variable(torch.tensor(X.values.astype(np.float32))).cuda(2)
X = X.join(runModel(Xv))

In [362]:
# Taken from Giba's topic : https://www.kaggle.com/titericz
# https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
# with Kyle Boone's post https://www.kaggle.com/kyleboone
classes = sorted(y.unique())
class_weights = {c: 1 for c in classes}
class_weights.update({c:2 for c in [64, 15]})
print('Unique classes : {}, {}'.format(len(classes), classes))
print(class_weights)

Unique classes : 14, [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
{6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}


## Train model with CV

In [287]:
lgbm_params = {'max_depth': 8,
 'n_estimators': 1024,
 'max_bin': 16,
 'num_leaves': 5,
 'feature_fraction': 0.5,
 'verbosity': 0}

In [395]:
eval_func = partial(
    lgbm_modeling_cross_validation, 
    X=X, 
    y=y, 
    classes=classes, 
    class_weights=class_weights, 
    nr_fold=5, 
    random_state=1
)

In [396]:
%%time
# modeling from CV
clfs, score = eval_func(lgbm_params)

Training until validation scores don't improve for 50 rounds.
[100]	training's multi_logloss: 0.441699	training's wloss: 0.430973	valid_1's multi_logloss: 0.817372	valid_1's wloss: 0.671012
Early stopping, best iteration is:
[118]	training's multi_logloss: 0.407722	training's wloss: 0.396876	valid_1's multi_logloss: 0.796825	valid_1's wloss: 0.667118
no 1-fold loss: 0.6671183407500729
Training until validation scores don't improve for 50 rounds.
[100]	training's multi_logloss: 0.445121	training's wloss: 0.435444	valid_1's multi_logloss: 0.820503	valid_1's wloss: 0.726302
Early stopping, best iteration is:
[120]	training's multi_logloss: 0.406731	training's wloss: 0.396353	valid_1's multi_logloss: 0.795336	valid_1's wloss: 0.722553
no 2-fold loss: 0.7225532219655024
Training until validation scores don't improve for 50 rounds.
[100]	training's multi_logloss: 0.446292	training's wloss: 0.437064	valid_1's multi_logloss: 0.80661	valid_1's wloss: 0.614217
[200]	training's multi_logloss: 0.3

In [ ]:
filename = 'subm_{:.6f}_{}.csv'.format(
    score, 
    dt.now().strftime('%Y-%m-%d-%H-%M')
)
print('save to {}'.format(filename))

## FINAL RUN

In [ ]:
%%time
# should take 100x (time after 1st iteration)
predict_test(
    clfs,
    features=X.columns, 
    output_path=filename,
    input_path='feat_test.csv',
    chunks=chunk_size_one_100,
    n_jobs=4
)

In [ ]:
z = pd.read_csv(filename)
print("Shape BEFORE grouping: {}".format(z.shape))
z = z.groupby('object_id').mean()
print("Shape AFTER grouping: {}".format(z.shape))
z.to_csv('single_{}'.format(filename), index=True)